# Xai _ Grad-Cam (TF)

In [16]:
import sys
sys.path.append('/Users/constance/Documents/Project_lung_cancer/src/Segmentation')
from TheDuneAI import ContourPilot as cp
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tqdm import tqdm

In [17]:
model_path = '/Users/constance/Documents/Project_lung_cancer/src/Segmentation/model_files/'
path_to_test_data = '/Users/constance/Documents/Project_lung_cancer/NIH dataset_raw/NRRD/converted_nrrds/'
save_path = '/Users/constance/Documents/Project_lung_cancer/NIH dataset_raw/Processed'

In [18]:
model = cp(model_path,path_to_test_data,save_path,verbosity=True)
for i, layer in enumerate(model.model1.layers):
        print(i, layer.name, layer.__class__.__name__)


0 input_1 InputLayer
1 conv2d_1 Conv2D
2 conv2d_2 Conv2D
3 max_pooling2d_1 MaxPooling2D
4 conv2d_3 Conv2D
5 conv2d_4 Conv2D
6 max_pooling2d_2 MaxPooling2D
7 conv2d_5 Conv2D
8 conv2d_6 Conv2D
9 max_pooling2d_3 MaxPooling2D
10 conv2d_7 Conv2D
11 conv2d_8 Conv2D
12 dropout_1 Dropout
13 max_pooling2d_4 MaxPooling2D
14 conv2d_9 Conv2D
15 conv2d_10 Conv2D
16 dropout_2 Dropout
17 up_sampling2d_1 UpSampling2D
18 conv2d_11 Conv2D
19 concatenate_1 Concatenate
20 conv2d_12 Conv2D
21 conv2d_13 Conv2D
22 up_sampling2d_2 UpSampling2D
23 conv2d_14 Conv2D
24 concatenate_2 Concatenate
25 conv2d_15 Conv2D
26 conv2d_16 Conv2D
27 up_sampling2d_3 UpSampling2D
28 conv2d_17 Conv2D
29 concatenate_3 Concatenate
30 conv2d_18 Conv2D
31 conv2d_19 Conv2D
32 up_sampling2d_4 UpSampling2D
33 conv2d_20 Conv2D
34 concatenate_4 Concatenate
35 conv2d_21 Conv2D
36 conv2d_22 Conv2D
37 conv2d_23 Conv2D
38 conv2d_24 Conv2D


100%|██████████| 299/299 [00:00<00:00, 9061.13it/s]

299  Patients found
Epoch end
------- Loading patients in predict mode -------
0 input_1 InputLayer
1 conv2d_1 Conv2D
2 conv2d_2 Conv2D
3 max_pooling2d_1 MaxPooling2D
4 conv2d_3 Conv2D
5 conv2d_4 Conv2D
6 max_pooling2d_2 MaxPooling2D
7 conv2d_5 Conv2D
8 conv2d_6 Conv2D
9 max_pooling2d_3 MaxPooling2D
10 conv2d_7 Conv2D
11 conv2d_8 Conv2D
12 dropout_1 Dropout
13 max_pooling2d_4 MaxPooling2D
14 conv2d_9 Conv2D
15 conv2d_10 Conv2D
16 dropout_2 Dropout
17 up_sampling2d_1 UpSampling2D
18 conv2d_11 Conv2D
19 concatenate_1 Concatenate
20 conv2d_12 Conv2D
21 conv2d_13 Conv2D
22 up_sampling2d_2 UpSampling2D
23 conv2d_14 Conv2D
24 concatenate_2 Concatenate
25 conv2d_15 Conv2D
26 conv2d_16 Conv2D
27 up_sampling2d_3 UpSampling2D
28 conv2d_17 Conv2D
29 concatenate_3 Concatenate
30 conv2d_18 Conv2D
31 conv2d_19 Conv2D
32 up_sampling2d_4 UpSampling2D
33 conv2d_20 Conv2D
34 concatenate_4 Concatenate
35 conv2d_21 Conv2D
36 conv2d_22 Conv2D
37 conv2d_23 Conv2D
38 conv2d_24 Conv2D


In [19]:
target_layer = model.model1.get_layer("conv2d_21")
filename = list(model.Patient_dict.keys())[0]

gen_with_progress = tqdm(model.Patients_gen, desc='Progress')
gen_iterator = iter(gen_with_progress)  # <-- transformer en itérateur
img, _, filename, params = next(gen_iterator)

Progress:   0%|          | 0/299 [00:22<?, ?it/s]

(44, 512, 512)


Window limits  -1000 150.0
Amount of unique values, original img:  1151 resampled img:  46


In [20]:
last_conv_layer_model = Model(inputs=model.model1.input,
                              outputs=target_layer.output)

x = tf.keras.Input(shape=target_layer.output.shape[1:]) 
y = x
for layer in model.model1.layers[model.model1.layers.index(target_layer) + 1:]:
    y = layer(y)
seg_model = Model(inputs=x, outputs=y)

In [21]:
def generate_segmentation_gradcam(model_obj, img, target_layer_name="conv2d_21", pixel_of_interest=None):
    """
    Génère une carte Grad-CAM pour un modèle de segmentation.
    
    Arguments :
      - model_obj : l'objet modèle tel que chargé via ContourPilot (utilise model_obj.model1 comme tf.keras.Model).
      - img       : image prétraitée (NumPy array) de forme (batch, H, W, channels). On suppose par ex. (1, 512, 512, 1).
      - target_layer_name : nom de la couche convolutionnelle à utiliser (par défaut "conv2d_21").
      - pixel_of_interest : tuple (i, j) optionnel pour sélectionner un pixel (h, w) dans la prédiction
                            sur lequel se baser pour calculer Grad-CAM. 
                            Par défaut, on prendra le pixel central.
    
    Renvoie :
      - heatmap   : la carte Grad-CAM sous forme de tableau NumPy (shape = (H_conv, W_conv)).
      - preds     : la prédiction du modèle final (utile pour vérifier la segmentation).
    """
    
    # Récupérer la couche cible
    target_layer = model_obj.model1.get_layer(target_layer_name)
    
    # Construire le sous-modèle jusqu'à la couche cible
    last_conv_layer_model = Model(inputs=model_obj.model1.input, outputs=target_layer.output)
    
    # Construire le "classifier" à partir de target_layer jusqu'à la sortie
    x = tf.keras.Input(shape=target_layer.output.shape[1:])
    y = x
    # On prend toutes les couches après target_layer dans model_obj.model1
    start_idx = model_obj.model1.layers.index(target_layer) + 1
    for layer in model_obj.model1.layers[start_idx:]:
        y = layer(y)
    classifier_model = Model(inputs=x, outputs=y)
    
    # Si aucun pixel d'intérêt n'est spécifié, prendre celui du centre de la prédiction
    with tf.GradientTape() as tape:
        # Passage dans le sous-modèle de la couche cible
        conv_output = last_conv_layer_model(img)  # forme : (1, H_conv, W_conv, channels)
        tape.watch(conv_output)
        
        preds = classifier_model(conv_output)  # sortie de la segmentation, par exemple (1, H_out, W_out, 1)
        
        # Choix du pixel de référence sur lequel calculer le gradient.
        # Pour la segmentation, il peut être pertinent de prendre le pixel central de la carte de prédiction
        if pixel_of_interest is None:
            # On suppose qu'on a une unique carte de segmentation (1 canal) et on prend le centre de la carte de prédiction
            H_out, W_out = preds.shape[1], preds.shape[2]
            i, j = H_out // 2, W_out // 2
        else:
            i, j = pixel_of_interest
        
        # Extraire la valeur (le score) de la prédiction en ce point
        # preds est de forme (1, H_out, W_out, channels); on prend par exemple le canal 0 si c'est binaire
        top_class_channel = preds[0, i, j, 0]
    
    # Calcul du gradient de ce score par rapport à la sortie de la couche cible
    grads = tape.gradient(top_class_channel, conv_output)
    
    # Calcul de la moyenne (pooled) sur les dimensions spatiales
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))   # forme : (channels,)
    
    # Extraire la sortie de la couche cible (pour le premier élément du batch)
    conv_output = conv_output[0]   # forme : (H_conv, W_conv, channels)
    
    # Faire une pondération canal par canal
    heatmap = conv_output * pooled_grads  # broadcast sur le canal
    heatmap = tf.reduce_sum(heatmap, axis=-1)  # somme pondérée sur les canaux
    
    # Appliquer ReLU et normaliser la heatmap
    heatmap = tf.nn.relu(heatmap)
    if tf.reduce_max(heatmap) > 0:
        heatmap /= tf.reduce_max(heatmap)
    heatmap = heatmap.numpy()
    
    return heatmap, preds


In [22]:
# Générer Grad-CAM
heatmap, preds = generate_segmentation_gradcam(model, img, target_layer_name="conv2d_21")

# Affichage : superposition de la heatmap sur la slice d'origine
plt.figure(figsize=(8, 8))
plt.imshow(img[..., 0], cmap="gray")
# Redimensionne la heatmap pour qu'elle corresponde à l'image d'origine
import cv2
heatmap_resized = cv2.resize(heatmap, (img.shape[2], img.shape[1]))
plt.imshow(heatmap_resized, cmap="jet", alpha=0.5)
plt.title("Grad-CAM Heatmap (Segmentation)")
plt.axis('off')
plt.show()

ValueError: Input 0 of layer "model_6" is incompatible with the layer: expected shape=(None, 512, 512, 1), found shape=(1, 38, 512, 512)